# Natural Gas Powerplants and Environmental Justice - Nationwide and in Maryland

## Linear and logistic regressions using a binary variable representing existence of a natural gas power plant in census tracts (not used in final analysis)

In [162]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [163]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [164]:
%%R

require('tidyverse')
require('DescTools')

In [165]:
df = pd.read_csv('plants_merged_eji.csv')
df.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,GEOID_2020,COUNTY_x,StateDesc,STATEABBR,LOCATION,E_TOTPOP,M_TOTPOP,E_DAYPOP,SPL_EJI,RPL_EJI,SPL_SER,RPL_SER,SPL_EJI_CBM,RPL_EJI_CBM,E_MINRTY,EPL_MINRTY,SPL_SVM_DOM1,RPL_SVM_DOM1,E_POV200,EPL_POV200,E_NOHSDP,EPL_NOHSDP,E_UNEMP,EPL_UNEMP,E_RENTER,EPL_RENTER,E_HOUBDN,EPL_HOUBDN,E_UNINSUR,EPL_UNINSUR,E_NOINT,EPL_NOINT,SPL_SVM_DOM2,RPL_SVM_DOM2,E_AGE65,EPL_AGE65,E_AGE17,EPL_AGE17,E_DISABL,EPL_DISABL,E_LIMENG,EPL_LIMENG,SPL_SVM_DOM3,RPL_SVM_DOM3,E_MOBILE,EPL_MOBILE,E_GROUPQ,EPL_GROUPQ,SPL_SVM_DOM4,RPL_SVM_DOM4,SPL_SVM,RPL_SVM,E_OZONE,EPL_OZONE,E_PM,EPL_PM,E_DSLPM,EPL_DSLPM,E_TOTCR,EPL_TOTCR,SPL_EBM_DOM1,RPL_EBM_DOM1,E_NPL,EPL_NPL,E_TRI,EPL_TRI,E_TSD,EPL_TSD,E_RMP,EPL_RMP,E_COAL,EPL_COAL,E_LEAD,EPL_LEAD,SPL_EBM_DOM2,RPL_EBM_DOM2,E_PARK,EPL_PARK,E_HOUAGE,EPL_HOUAGE,E_WLKIND,EPL_WLKIND,SPL_EBM_DOM3,RPL_EBM_DOM3,E_RAIL,EPL_RAIL,E_ROAD,EPL_ROAD,E_AIRPRT,EPL_AIRPRT,SPL_EBM_DOM4,RPL_EBM_DOM4,E_IMPWTR,EPL_IMPWTR,F_IMPWTR,SPL_EBM_DOM5,RPL_EBM_DOM5,SPL_EBM,RPL_EBM,E_ASTHMA,EPL_ASTHMA,F_ASTHMA,E_CANCER,EPL_CANCER,F_CANCER,E_CHD,EPL_CHD,F_CHD,E_MHLTH,EPL_MHLTH,F_MHLTH,E_DIABETES,EPL_DIABETES,F_DIABETES,F_HVM,RPL_HVM,E_NEHD,EPL_NEHD,SPL_CBM_DOM1,RPL_CBM_DOM1,E_BURN,EPL_BURN,E_SMOKE,EPL_SMOKE,SPL_CBM_DOM2,RPL_CBM_DOM2,E_CFLD,EPL_CFLD,F_CFLD,E_DRGT,EPL_DRGT,F_DRGT,E_HRCN,EPL_HRCN,F_HRCN,E_RFLD,EPL_RFLD,F_RFLD,E_SWND,EPL_SWND,F_SWND,E_TRND,EPL_TRND,F_TRND,SPL_CBM_DOM3,RPL_CBM_DOM3,SPL_CBM,RPL_CBM,AFAM,E_AFAM,HISP,E_HISP,ASIAN,E_ASIAN,AIAN,E_AIAN,NHPI,E_NHPI,TWOMORE,E_TWOMORE,OTHERRACE,E_OTHERRACE,Tribe_PCT_Tract,Tribe_Names,Tribe_Flag,Unnamed: 0.1,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,STATE,COUNTY_y,TRACT,contains_energy_source,Technology_Category,Category_Number,fossil_fuels,renewables,energy_category_numeric
0,10.0,1.0,40100.0,140000US10001040100,1.000104e+10,1.000104e+10,Kent County,Delaware,DE,Census Tract 401; Kent County; Delaware,7343.0,882.0,2855.0,0.9588,0.3030,0.7588,0.3220,1.7110,0.4451,18.8,0.3142,0.3142,0.3142,22.4347,0.3828,10.2,0.5751,6.4,0.675,14.0,0.2018,20.5,0.3654,3.6,0.2498,7.6,0.3606,2.8105,0.3421,14.3,0.3855,27.0,0.8032,14.1,0.5907,2.4,0.6287,2.4081,0.8160,22.9,0.9137,0.0,0.0000,0.9137,0.5878,6.4465,0.4406,0.365,0.6153,0.0,0.0,0.1182,0.2124,20.0,0.0434,0.8711,0.2485,0.1812,0.9257,0.0000,0.0000,0.0,0.0,0.0000,0.0000,0.0,0.0,0.0,0.0,0.9257,0.4162,99.9998,0.5517,23.0,0.1834,3.4444,0.9748,1.7099,0.5709,0.0000,0.0000,4.9654,0.1320,0.9892,0.8372,0.9692,0.4497,0.606,0.5750,0.0,0.5750,0.5750,5.0509,0.3182,12.2,0.8686,1.0,9.0,0.6320,0.0,7.3,0.5726,0.0,17.4,0.5557,0.0,12.8,0.5925,0.0,1.0,0.2,8.4,0.1951,0.1951,0.1951,0.0,0.0,7.1,0.3827,0.3827,0.3796,0.010,0.0360,9.0,5.7273,0.3153,9.0,0.1052,0.6608,9.0,1.5417,0.3903,9.0,5.288,0.7768,9.0,0.0556,0.5198,9.0,2.6990,0.8823,3.2768,0.7522,505.0,6.9,509.0,6.9,107.0,1.5,4.0,0.1,0.0,0.0,243.0,3.3,13.0,0.2,0.056753,Lenape Indian Tribe of Delaware SDTSA,0.0,1252.0,1252.0,Blue Hen Solar,65791.0,DE,IPP Non-CHP,PV,SUN,Solar Photovoltaic,NaN,NaN,NaN,Renewable Energy Sources,"map/?center=-75.68538,39.18568&level=14",-75.685380,39.185680,10.0,1.0,40100.0,1,Solar,1,0,1,1.0
1,10.0,1.0,40201.0,140000US10001040201,1.000104e+10,1.000104e+10,Kent County,Delaware,DE,Census Tract 402.01; Kent County; Delaware,5132.0,524.0,9054.0,1.1657,0.4323,0.9657,0.4892,1.7197,0.4501,40.8,0.5834,0.5834,0.5834,16.8532,0.2570,5.4,0.3154,7.4,0.747,31.5,0.5269,17.7,0.2628,6.4,0.4713,4.1,0.1766,2.7570,0.3306,11.0,0.2173,28.1,0.8449,13.0,0.5203,0.0,0.0000,1.5825,0.2011,4.8,0.6991,0.1,0.3697,1.0688,0.7033,5.9917,0.3636,0.365,0.6153,0.0,0.0,0.1646,0.3494,20.0,0.0434,1.0081,0.3095,41.2434,0.9764,82.8961,0.7699,0.0,0.0,58.3930,0.9152,0.0,0.0,0.0,0.0,2.6615,0.9474,100.0000,0.5514,37.5,0.3079,7.4583,0.6182,1.4775,0.3313,64.3592,0.6576,7.9819,0.1514,0.0000,0.0000,0.8090,0.3777,0.333,0.4021,0.0,0.4021,0

In [97]:
df.shape

(90173, 195)

In [98]:
df['natural_gas'] = (df['Category_Number'] == 2).astype(int) 

In [99]:
df['no_powerplant'] = (df['Category_Number'] == 0).astype(int) 

In [148]:
import numpy as np

df['natgas_noplant'] = np.where(df['Category_Number'] == 2, 1, 
                              np.where(df['Category_Number'] == 0, 0, np.nan))

In [149]:
df['natgas_noplant'].value_counts()

natgas_noplant
0.0    77512
1.0     1602
Name: count, dtype: int64

In [ ]:
## NatGas tracts v. tracts without any power type

In [31]:
## NatGas tracts v. tracts in same counties 

In [100]:
pd.set_option('display.max_columns', None)

In [101]:
df[df['Category_Number'] == 2]

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,GEOID_2020,COUNTY_x,StateDesc,STATEABBR,LOCATION,E_TOTPOP,M_TOTPOP,E_DAYPOP,SPL_EJI,RPL_EJI,SPL_SER,RPL_SER,SPL_EJI_CBM,RPL_EJI_CBM,E_MINRTY,EPL_MINRTY,SPL_SVM_DOM1,RPL_SVM_DOM1,E_POV200,EPL_POV200,E_NOHSDP,EPL_NOHSDP,E_UNEMP,EPL_UNEMP,E_RENTER,EPL_RENTER,E_HOUBDN,EPL_HOUBDN,E_UNINSUR,EPL_UNINSUR,E_NOINT,EPL_NOINT,SPL_SVM_DOM2,RPL_SVM_DOM2,E_AGE65,EPL_AGE65,E_AGE17,EPL_AGE17,E_DISABL,EPL_DISABL,E_LIMENG,EPL_LIMENG,SPL_SVM_DOM3,RPL_SVM_DOM3,E_MOBILE,EPL_MOBILE,E_GROUPQ,EPL_GROUPQ,SPL_SVM_DOM4,RPL_SVM_DOM4,SPL_SVM,RPL_SVM,E_OZONE,EPL_OZONE,E_PM,EPL_PM,E_DSLPM,EPL_DSLPM,E_TOTCR,EPL_TOTCR,SPL_EBM_DOM1,RPL_EBM_DOM1,E_NPL,EPL_NPL,E_TRI,EPL_TRI,E_TSD,EPL_TSD,E_RMP,EPL_RMP,E_COAL,EPL_COAL,E_LEAD,EPL_LEAD,SPL_EBM_DOM2,RPL_EBM_DOM2,E_PARK,EPL_PARK,E_HOUAGE,EPL_HOUAGE,E_WLKIND,EPL_WLKIND,SPL_EBM_DOM3,RPL_EBM_DOM3,E_RAIL,EPL_RAIL,E_ROAD,EPL_ROAD,E_AIRPRT,EPL_AIRPRT,SPL_EBM_DOM4,RPL_EBM_DOM4,E_IMPWTR,EPL_IMPWTR,F_IMPWTR,SPL_EBM_DOM5,RPL_EBM_DOM5,SPL_EBM,RPL_EBM,E_ASTHMA,EPL_ASTHMA,F_ASTHMA,E_CANCER,EPL_CANCER,F_CANCER,E_CHD,EPL_CHD,F_CHD,E_MHLTH,EPL_MHLTH,F_MHLTH,E_DIABETES,EPL_DIABETES,F_DIABETES,F_HVM,RPL_HVM,E_NEHD,EPL_NEHD,SPL_CBM_DOM1,RPL_CBM_DOM1,E_BURN,EPL_BURN,E_SMOKE,EPL_SMOKE,SPL_CBM_DOM2,RPL_CBM_DOM2,E_CFLD,EPL_CFLD,F_CFLD,E_DRGT,EPL_DRGT,F_DRGT,E_HRCN,EPL_HRCN,F_HRCN,E_RFLD,EPL_RFLD,F_RFLD,E_SWND,EPL_SWND,F_SWND,E_TRND,EPL_TRND,F_TRND,SPL_CBM_DOM3,RPL_CBM_DOM3,SPL_CBM,RPL_CBM,AFAM,E_AFAM,HISP,E_HISP,ASIAN,E_ASIAN,AIAN,E_AIAN,NHPI,E_NHPI,TWOMORE,E_TWOMORE,OTHERRACE,E_OTHERRACE,Tribe_PCT_Tract,Tribe_Names,Tribe_Flag,Unnamed: 0.1,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,STATE,COUNTY_y,TRACT,contains_energy_source,Technology_Category,Category_Number,fossil_fuels,renewables,energy_category_numeric,natural_gas,no_powerplant
4,10.0,1.0,40204.0,140000US10001040204,1.000104e+10,1.000104e+10,Kent County,Delaware,DE,Census Tract 402.04; Kent County; Delaware,6265.0,689.0,5555.0,1.1357,0.4135,0.9357,0.4646,1.6385,0.4038,47.4,0.6419,0.6419,0.6419,15.3948,0.2241,7.3,0.4314,6.6,0.6900,13.5,0.1911,16.3,0.2127,4.6,0.3346,5.0,0.2243,2.3082,0.2358,15.5,0.4520,22.8,0.5752,10.3,0.3319,0.6,0.3756,1.7347,0.2940,2.6,0.6358,2.9,0.8370,1.4728,0.8695,6.1576,0.3910,0.365,0.6153,0.0,0.0,0.1687,0.3610,20.0,0.0434,1.0197,0.3145,50.5217,0.9805,59.8171,0.6940,0.0000,0.0000,10.8851,0.7861,0.0,0.0,0.0,0.0,2.4606,0.8920,100.0,0.5514,32.8,0.2643,9.7500,0.4372,1.2529,0.1565,73.6904,0.6882,27.2971,0.2658,0.0,0.0,0.9540,0.4405,0.333,0.4021,0.0,0.4021,0.4021,6.0893,0.5447,11.8,0.8079,1.0,7.2,0.3903,0.0,5.8,0.2866,0.0,16.3,0.4226,0.0,12.0,0.4985,0.0,1.0,0.2,8.6,0.2039,0.2039,0.2039,0.0,0.0,7.0,0.3651,0.3651,0.3621,0.010,0.0360,9.0,5.7273,0.3153,9.0,0.1052,0.6608,9.0,1.5417,0.3903,9.0,5.2880,0.7768,9.0,0.0278,0.0000,2.0,2.1792,0.6760,2.7482,0.5028,1910.0,30.5,249.0,4.0,113.0,1.8,0.0,0.0,0.0,0.0,696.0,11.1,0.0,0.0,0.0,-999,NaN,11799.0,11799.0,Warren F Sam Beasley Generation Station,7962.0,DE,Electric Utility,GT,NG,Natural Gas Fired Combustion Turbine,NaN,NaN,NaN,Natural Gas,"map/?center=-75.623928,39.279179&level=14",-75.623928,39.279179,10.0,1.0,40204.0,1,Natural Gas,2,1,0,2.0,1,0
9,10.0,1.0,40502.0,140000US10001040502,1.000104e+10,1.000104e+10,Kent County,Delaware,DE,Census Tract 405.02; Kent County; Delaware,2647.0,451.0,869.0,1.8191,0.7758,1.0191,0.5332,2.7820,0.9257,52.3,0.6796,0.6796,0.6796,24.4805,0.4273,5.0,0.2895,11.2,0.9009,28.3,0.4773,32.2,0.7147,11.4,0.7294,9.2,0.4408,3.9799,0.6170,18.1,0.5962,22.5,0.5562,15.6,0.6776,0.6,0.3756,2.2056,0.6795,24.8,0.9247,0.0,0.0000,0.9247,0.5997,7.7898,0.6738,0.274,0.5463,0.0,0.0,0.1518,0.3135,20.0,0.0434,0.9032,0.2621,5.4548,0.9463,16.0854,0.5125,0.0000,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,1.4588,0.6022,100.0,0.5514,22.1,0.1772,4.0625,0.9436,1.6722,0.5341,1.6728,0.3486,49.2025,0.3579,0.0,0.

In [102]:
df['county'] = df['COUNTY_x']

In [103]:
df['natural_gas'].value_counts()

natural_gas
0    88571
1     1602
Name: count, dtype: int64

In [104]:
%%R

install.packages('fixest')


The downloaded binary packages are in
	/var/folders/mb/f5zh4qyd6sbf171lrlnmff040000gn/T//Rtmprm8QrK/downloaded_packages


trying URL 'https://cran.case.edu/bin/macosx/big-sur-arm64/contrib/4.4/fixest_0.12.1.tgz'
Content type 'application/x-gzip' length 5328506 bytes (5.1 MB)
downloaded 5.1 MB



In [105]:
%%R

library(fixest)

In [150]:
%%R -i df
# Logistic regression with county fixed effects
model <- feglm(
  natgas_noplant ~ E_POV200 + E_AFAM | county, 
  data = df, 
  family = binomial("logit")
)

summary(model)

GLM estimation, family = binomial, Dep. Var.: natgas_noplant
Observations: 55,166
Fixed-effects: county: 532
Standard-errors: Clustered (county) 
          Estimate Std. Error  z value  Pr(>|z|)    
E_POV200  0.006217   0.002314  2.68722 0.0072049 ** 
E_AFAM   -0.006576   0.002403 -2.73657 0.0062083 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Log-Likelihood: -6,173.3   Adj. Pseudo R2: 0.0257  
           BIC: 18,176.8     Squared Cor.: 0.045692


NOTES: 13,249 observations removed because of NA values (LHS: 11,059, RHS: 2,704, Fixed-effects: 167).
       1,249 fixed-effects (21,758 observations) removed because of only 0 (or only 1) outcomes.


In [107]:
%%R -i df
logistic <- glm(natural_gas ~ E_POV200 + E_AFAM, data=df, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ E_POV200 + E_AFAM, family = "binomial", 
    data = df)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.232298   0.053104 -79.698  < 2e-16 ***
E_POV200     0.009831   0.001521   6.462 1.04e-10 ***
E_AFAM      -0.010135   0.001588  -6.383 1.74e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 15277  on 87468  degrees of freedom
Residual deviance: 15214  on 87466  degrees of freedom
  (2704 observations deleted due to missingness)
AIC: 15220

Number of Fisher Scoring iterations: 7



In [108]:
%%R

require('DescTools')

PseudoR2(logistic, which='McFadden')

   McFadden 
0.004135378 


In [109]:
%%R -i df
logistic <- glm(natural_gas ~ E_POV200 + E_AFAM, data=df, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ E_POV200 + E_AFAM, family = "binomial", 
    data = df)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.232298   0.053104 -79.698  < 2e-16 ***
E_POV200     0.009831   0.001521   6.462 1.04e-10 ***
E_AFAM      -0.010135   0.001588  -6.383 1.74e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 15277  on 87468  degrees of freedom
Residual deviance: 15214  on 87466  degrees of freedom
  (2704 observations deleted due to missingness)
AIC: 15220

Number of Fisher Scoring iterations: 7



In [110]:
%%R

PseudoR2(logistic, which='McFadden')

   McFadden 
0.004135378 


In [111]:
%%R -i df
logistic <- glm(natural_gas ~ E_POV200 + E_AFAM + E_HISP + E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + E_ROAD + E_RAIL + E_AIRPRT + E_IMPWTR + E_SMOKE + E_NOINT + E_HOUBDN + E_RENTER + E_DAYPOP, data=df, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ E_POV200 + E_AFAM + E_HISP + E_AIAN + 
    E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + 
    E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + E_ROAD + E_RAIL + 
    E_AIRPRT + E_IMPWTR + E_SMOKE + E_NOINT + E_HOUBDN + E_RENTER + 
    E_DAYPOP, family = "binomial", data = df)

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -3.850e+00  2.074e-01 -18.564  < 2e-16 ***
E_POV200     -7.764e-03  2.982e-03  -2.604  0.00922 ** 
E_AFAM       -2.409e-03  1.910e-03  -1.261  0.20716    
E_HISP        1.200e-02  1.927e-03   6.228 4.74e-10 ***
E_AIAN       -2.223e-02  8.945e-03  -2.485  0.01296 *  
E_LIMENG     -1.856e-02  7.366e-03  -2.520  0.01174 *  
SPL_SVM_DOM4  4.592e-01  6.193e-02   7.416 1.21e-13 ***
E_UNEMP       1.346e-02  6.409e-03   2.099  0.03579 *  
E_NOHSDP      1.784e-03  4.795e-03   0.372  0.70988    
E_UNINSUR     4.033e-03  4.466e-03   0.903  0.36653    
E_TOTPOP      2.028e-05  1.520e-05   1.334  0.18205  

In [112]:
%%R

PseudoR2(logistic, which='McFadden')

  McFadden 
0.07098704 


In [113]:
%%R -i df
model <- lm(E_TOTCR ~ natural_gas, data=df)
summary(model)


Call:
lm(formula = E_TOTCR ~ natural_gas, data = df)

Residuals:
   Min     1Q Median     3Q    Max 
-18.29  -5.29  -5.29   4.71 374.71 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 25.29480    0.02832 893.122   <2e-16 ***
natural_gas  0.18279    0.21423   0.853    0.394    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 8.331 on 88054 degrees of freedom
  (2117 observations deleted due to missingness)
Multiple R-squared:  8.267e-06,	Adjusted R-squared:  -3.089e-06 
F-statistic: 0.728 on 1 and 88054 DF,  p-value: 0.3935



In [114]:
%%R -i df
model <- lm(E_TOTCR ~ fossil_fuels, data=df)
summary(model)


Call:
lm(formula = E_TOTCR ~ fossil_fuels, data = df)

Residuals:
   Min     1Q Median     3Q    Max 
-18.33  -5.33  -4.21   4.67 374.67 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  25.32795    0.02845 890.282  < 2e-16 ***
fossil_fuels -1.12098    0.17404  -6.441 1.19e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 8.329 on 88054 degrees of freedom
  (2117 observations deleted due to missingness)
Multiple R-squared:  0.0004709,	Adjusted R-squared:  0.0004596 
F-statistic: 41.49 on 1 and 88054 DF,  p-value: 1.193e-10



In [115]:
%%R -i df
model <- lm(E_TOTCR ~ natural_gas + E_POV200 + E_AFAM, data=df)
summary(model)


Call:
lm(formula = E_TOTCR ~ natural_gas + E_POV200 + E_AFAM, data = df)

Residuals:
   Min     1Q Median     3Q    Max 
-18.45  -4.30  -3.66   5.43 375.81 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 24.291127   0.054343 446.993   <2e-16 ***
natural_gas  0.376493   0.208270   1.808   0.0707 .  
E_POV200    -0.014807   0.001667  -8.884   <2e-16 ***
E_AFAM       0.114037   0.001441  79.126   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 8.028 on 87458 degrees of freedom
  (2711 observations deleted due to missingness)
Multiple R-squared:  0.07221,	Adjusted R-squared:  0.07217 
F-statistic:  2269 on 3 and 87458 DF,  p-value: < 2.2e-16



In [116]:
%%R -i df
model <- lm(E_CANCER ~ natural_gas, data=df)
summary(model)


Call:
lm(formula = E_CANCER ~ natural_gas, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-7.4878 -1.9878  0.1122  1.7122 19.5122 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  7.987846   0.009147  873.32  < 2e-16 ***
natural_gas -0.529573   0.069500   -7.62 2.57e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.683 on 87584 degrees of freedom
  (2587 observations deleted due to missingness)
Multiple R-squared:  0.0006625,	Adjusted R-squared:  0.0006511 
F-statistic: 58.06 on 1 and 87584 DF,  p-value: 2.566e-14



In [117]:
%%R -i df
model <- lm(E_CANCER ~ natural_gas + E_POV200 + E_AFAM + E_HISP + E_UNEMP + E_PM + E_OZONE + E_NOHSDP + E_UNINSUR + E_GROUPQ + E_TOTPOP + E_AGE65 + E_AGE17, data=df)
summary(model)


Call:
lm(formula = E_CANCER ~ natural_gas + E_POV200 + E_AFAM + E_HISP + 
    E_UNEMP + E_PM + E_OZONE + E_NOHSDP + E_UNINSUR + E_GROUPQ + 
    E_TOTPOP + E_AGE65 + E_AGE17, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-17.7209  -0.6139   0.0601   0.6743  11.5599 

Coefficients:
              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  4.293e+00  2.335e-02  183.838  < 2e-16 ***
natural_gas -3.893e-02  2.851e-02   -1.365  0.17214    
E_POV200    -9.770e-03  3.181e-04  -30.716  < 2e-16 ***
E_AFAM      -3.780e-02  2.137e-04 -176.914  < 2e-16 ***
E_HISP      -3.398e-02  2.544e-04 -133.567  < 2e-16 ***
E_UNEMP      2.454e-03  9.304e-04    2.638  0.00835 ** 
E_PM        -6.034e-02  4.893e-03  -12.330  < 2e-16 ***
E_OZONE     -4.485e-03  1.740e-03   -2.578  0.00995 ** 
E_NOHSDP    -1.526e-02  6.178e-04  -24.705  < 2e-16 ***
E_UNINSUR    1.934e-02  6.619e-04   29.221  < 2e-16 ***
E_GROUPQ    -1.112e-02  4.973e-04  -22.358  < 2e-16 ***
E_TOTPOP    -2.120e-05 

In [118]:
%%R -i df
model <- lm(E_ASTHMA ~ natural_gas, data=df)
summary(model)


Call:
lm(formula = E_ASTHMA ~ natural_gas, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.6321 -0.9321 -0.0521  0.8679 10.0679 

Coefficients:
             Estimate Std. Error  t value Pr(>|t|)    
(Intercept) 10.632080   0.004878 2179.452   <2e-16 ***
natural_gas -0.080004   0.037068   -2.158   0.0309 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.431 on 87584 degrees of freedom
  (2587 observations deleted due to missingness)
Multiple R-squared:  5.318e-05,	Adjusted R-squared:  4.177e-05 
F-statistic: 4.658 on 1 and 87584 DF,  p-value: 0.03091



In [119]:
%%R -i df
model <- lm(E_ASTHMA ~ natural_gas + E_POV200 + E_AFAM + E_HISP + E_UNEMP + E_PM + E_OZONE + E_NOHSDP + E_UNINSUR + E_GROUPQ + E_TOTPOP + E_AGE65 + E_AGE17, data=df)
summary(model)


Call:
lm(formula = E_ASTHMA ~ natural_gas + E_POV200 + E_AFAM + E_HISP + 
    E_UNEMP + E_PM + E_OZONE + E_NOHSDP + E_UNINSUR + E_GROUPQ + 
    E_TOTPOP + E_AGE65 + E_AGE17, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.2924 -0.6146 -0.0292  0.5942  6.2447 

Coefficients:
              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  9.266e+00  2.114e-02  438.386  < 2e-16 ***
natural_gas -8.584e-02  2.580e-02   -3.326 0.000880 ***
E_POV200     4.826e-02  2.879e-04  167.629  < 2e-16 ***
E_AFAM       5.704e-03  1.934e-04   29.500  < 2e-16 ***
E_HISP      -2.538e-02  2.302e-04 -110.234  < 2e-16 ***
E_UNEMP      2.027e-02  8.420e-04   24.067  < 2e-16 ***
E_PM         5.778e-02  4.429e-03   13.046  < 2e-16 ***
E_OZONE      5.486e-03  1.575e-03    3.484 0.000495 ***
E_NOHSDP     2.510e-02  5.591e-04   44.896  < 2e-16 ***
E_UNINSUR   -2.296e-02  5.990e-04  -38.331  < 2e-16 ***
E_GROUPQ    -5.760e-03  4.501e-04  -12.797  < 2e-16 ***
E_TOTPOP    -1.830e-05  2.123e-06

In [120]:
%%R -i df
model <- lm(E_CHD ~ natural_gas, data=df)
summary(model)


Call:
lm(formula = E_CHD ~ natural_gas, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-6.6811 -1.4811 -0.1811  1.3189 30.0189 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  7.081127   0.007468 948.209   <2e-16 ***
natural_gas -0.067218   0.056744  -1.185    0.236    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 2.191 on 87584 degrees of freedom
  (2587 observations deleted due to missingness)
Multiple R-squared:  1.602e-05,	Adjusted R-squared:  4.604e-06 
F-statistic: 1.403 on 1 and 87584 DF,  p-value: 0.2362



In [121]:
%%R -i df
model <- lm(E_CHD ~ natural_gas + E_POV200 + E_AFAM + E_HISP + E_UNEMP + E_PM + E_OZONE + E_NOHSDP + E_UNINSUR + E_GROUPQ + E_TOTPOP + E_AGE65 + E_AGE17, data=df)
summary(model)


Call:
lm(formula = E_CHD ~ natural_gas + E_POV200 + E_AFAM + E_HISP + 
    E_UNEMP + E_PM + E_OZONE + E_NOHSDP + E_UNINSUR + E_GROUPQ + 
    E_TOTPOP + E_AGE65 + E_AGE17, data = df)

Residuals:
     Min       1Q   Median       3Q      Max 
-11.7316  -0.6976   0.0093   0.7045  20.9848 

Coefficients:
              Estimate Std. Error  t value Pr(>|t|)    
(Intercept)  1.126e+00  2.447e-02   46.002  < 2e-16 ***
natural_gas  1.859e-01  2.988e-02    6.222 4.92e-10 ***
E_POV200     5.166e-02  3.333e-04  155.015  < 2e-16 ***
E_AFAM      -1.386e-02  2.239e-04  -61.927  < 2e-16 ***
E_HISP      -2.676e-02  2.665e-04 -100.383  < 2e-16 ***
E_UNEMP      1.317e-02  9.749e-04   13.513  < 2e-16 ***
E_PM        -1.776e-01  5.127e-03  -34.642  < 2e-16 ***
E_OZONE     -1.938e-02  1.823e-03  -10.629  < 2e-16 ***
E_NOHSDP     4.638e-02  6.473e-04   71.648  < 2e-16 ***
E_UNINSUR    2.680e-02  6.935e-04   38.641  < 2e-16 ***
E_GROUPQ    -1.071e-02  5.211e-04  -20.545  < 2e-16 ***
E_TOTPOP    -5.191e-05  2.

In [122]:
%%R -i df
model <- lm(E_PM ~ natural_gas, data=df)
summary(model)


Call:
lm(formula = E_PM ~ natural_gas, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.4620 -0.3176 -0.3176 -0.1356  9.6224 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.317583   0.002762 114.970  < 2e-16 ***
natural_gas 0.144450   0.020860   6.925 4.39e-12 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8132 on 88216 degrees of freedom
  (1955 observations deleted due to missingness)
Multiple R-squared:  0.0005433,	Adjusted R-squared:  0.000532 
F-statistic: 47.95 on 1 and 88216 DF,  p-value: 4.393e-12



In [123]:
%%R -i df
model <- lm(E_PM ~ natural_gas + E_POV200 + E_AFAM, data=df)
summary(model)


Call:
lm(formula = E_PM ~ natural_gas + E_POV200 + E_AFAM, data = df)

Residuals:
    Min      1Q  Median      3Q     Max 
-0.6747 -0.3608 -0.2809 -0.0460  9.5244 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.3328735  0.0054532  61.042  < 2e-16 ***
natural_gas  0.1258127  0.0209026   6.019 1.76e-09 ***
E_POV200     0.0021605  0.0001672  12.919  < 2e-16 ***
E_AFAM      -0.0063268  0.0001446 -43.747  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.8057 on 87465 degrees of freedom
  (2704 observations deleted due to missingness)
Multiple R-squared:  0.02214,	Adjusted R-squared:  0.0221 
F-statistic:   660 on 3 and 87465 DF,  p-value: < 2.2e-16



# Now focusing on Maryland alone

In [151]:
md = df[df['StateDesc'] == 'Maryland']

In [166]:
pd.set_option('display.max_columns', None)

In [167]:
md[md['natural_gas'] == 1]

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,GEOID_2020,COUNTY_x,StateDesc,STATEABBR,LOCATION,E_TOTPOP,M_TOTPOP,E_DAYPOP,SPL_EJI,RPL_EJI,SPL_SER,RPL_SER,SPL_EJI_CBM,RPL_EJI_CBM,E_MINRTY,EPL_MINRTY,SPL_SVM_DOM1,RPL_SVM_DOM1,E_POV200,EPL_POV200,E_NOHSDP,EPL_NOHSDP,E_UNEMP,EPL_UNEMP,E_RENTER,EPL_RENTER,E_HOUBDN,EPL_HOUBDN,E_UNINSUR,EPL_UNINSUR,E_NOINT,EPL_NOINT,SPL_SVM_DOM2,RPL_SVM_DOM2,E_AGE65,EPL_AGE65,E_AGE17,EPL_AGE17,E_DISABL,EPL_DISABL,E_LIMENG,EPL_LIMENG,SPL_SVM_DOM3,RPL_SVM_DOM3,E_MOBILE,EPL_MOBILE,E_GROUPQ,EPL_GROUPQ,SPL_SVM_DOM4,RPL_SVM_DOM4,SPL_SVM,RPL_SVM,E_OZONE,EPL_OZONE,E_PM,EPL_PM,E_DSLPM,EPL_DSLPM,E_TOTCR,EPL_TOTCR,SPL_EBM_DOM1,RPL_EBM_DOM1,E_NPL,EPL_NPL,E_TRI,EPL_TRI,E_TSD,EPL_TSD,E_RMP,EPL_RMP,E_COAL,EPL_COAL,E_LEAD,EPL_LEAD,SPL_EBM_DOM2,RPL_EBM_DOM2,E_PARK,EPL_PARK,E_HOUAGE,EPL_HOUAGE,E_WLKIND,EPL_WLKIND,SPL_EBM_DOM3,RPL_EBM_DOM3,E_RAIL,EPL_RAIL,E_ROAD,EPL_ROAD,E_AIRPRT,EPL_AIRPRT,SPL_EBM_DOM4,RPL_EBM_DOM4,E_IMPWTR,EPL_IMPWTR,F_IMPWTR,SPL_EBM_DOM5,RPL_EBM_DOM5,SPL_EBM,RPL_EBM,E_ASTHMA,EPL_ASTHMA,F_ASTHMA,E_CANCER,EPL_CANCER,F_CANCER,E_CHD,EPL_CHD,F_CHD,E_MHLTH,EPL_MHLTH,F_MHLTH,E_DIABETES,EPL_DIABETES,F_DIABETES,F_HVM,RPL_HVM,E_NEHD,EPL_NEHD,SPL_CBM_DOM1,RPL_CBM_DOM1,E_BURN,EPL_BURN,E_SMOKE,EPL_SMOKE,SPL_CBM_DOM2,RPL_CBM_DOM2,E_CFLD,EPL_CFLD,F_CFLD,E_DRGT,EPL_DRGT,F_DRGT,E_HRCN,EPL_HRCN,F_HRCN,E_RFLD,EPL_RFLD,F_RFLD,E_SWND,EPL_SWND,F_SWND,E_TRND,EPL_TRND,F_TRND,SPL_CBM_DOM3,RPL_CBM_DOM3,SPL_CBM,RPL_CBM,AFAM,E_AFAM,HISP,E_HISP,ASIAN,E_ASIAN,AIAN,E_AIAN,NHPI,E_NHPI,TWOMORE,E_TWOMORE,OTHERRACE,E_OTHERRACE,Tribe_PCT_Tract,Tribe_Names,Tribe_Flag,Unnamed: 0.1,Unnamed: 0,plant_name,Plant Code,State,Sector Name,Prime Movers,Fuel Types,Primary Technology,withdrawal_volume_million_gallons,discharge_volume_million_gallons,consumption_volume_million_gallons,energy_category,map_link,longitude,latitude,STATE,COUNTY_y,TRACT,contains_energy_source,Technology_Category,Category_Number,fossil_fuels,renewables,energy_category_numeric,natural_gas,no_powerplant,county,natgas_noplant
21794,24.0,15.0,31400.0,140000US24015031400,2.401503e+10,2.401503e+10,Cecil County,Maryland,MD,Census Tract 314; Cecil County; Maryland,3947.0,459.0,2090.0,1.3320,0.5302,0.7320,0.3016,1.9623,0.5876,10.4,0.1703,0.1703,0.1703,18.1633,0.2860,11.7,0.6345,6.6,0.6900,12.4,0.1684,20.5,0.3654,2.2,0.1314,9.2,0.4408,2.7165,0.3216,20.2,0.6996,18.5,0.3069,15.5,0.6721,0.0,0.0000,1.6786,0.2565,12.4,0.8218,0.4,0.5495,1.3713,0.8196,5.9367,0.3548,0.183,0.4757,0.091,0.5886,0.1680,0.3588,20.0,0.0434,1.4665,0.4965,0.0000,0.0000,0.0000,0.0000,18.9388,0.6001,0.0000,0.0000,0.0,0.0,0.0,0.0,0.6001,0.2687,93.4334,0.6296,30.4,0.2435,4.7500,0.8933,1.7664,0.6240,44.7513,0.5882,64.8388,0.4155,0.0000,0.0000,1.0037,0.4671,0.457,0.4853,0.0,0.4853,0.4853,5.3220,0.3772,11.6,0.7701,1.0,9.3,0.6799,1.0,8.1,0.7008,1.0,17.7,0.5899,0.0,13.2,0.6346,0.0,3.0,0.6,6.6,0.1146,0.1146,0.1146,0.0,0.0,6.3,0.2945,0.2945,0.2919,0.010,0.0360,9.0,7.3182,0.3724,9.0,0.0760,0.5691,9.0,2.6250,0.6608,9.0,7.4622,0.9596,9.0,0.0000,0.0000,0.0,2.5979,0.8406,3.0070,0.6303,100.0,2.5,111.0,2.8,0.0,0.0,13.0,0.3,0.0,0.0,185.0,4.7,0.0,0.0,0.0,-999,NaN,3632.0,3632.0,Essential Power Rock Springs LLC,7835.0,MD,IPP Non-CHP,GT,NG,Natural Gas Fired Combustion Turbine,NaN,NaN,NaN,Natural Gas,"map/?center=-76.15976,39.71901&level=14",-76.159760,39.719010,24.0,15.0,31400.0,1,Natural Gas,2,1,0,2.0,1,0,Cecil County,1.0
21795,24.0,15.0,31400.0,140000US24015031400,2.401503e+10,2.401503e+10,Cecil County,Maryland,MD,Census Tract 314; Cecil County; Maryland,3947.0,459.0,2090.0,1.3320,0.5302,0.7320,0.3016,1.9623,0.5876,10.4,0.1703,0.1703,0.1703,18.1633,0.2860,11.7,0.6345,6.6,0.6900,12.4,0.1684,20.5,0.3654,2.2,0.1314,9.2,0.4408,2.7165,0.3216,20.2,0.6996,18.5,0.3069,15.5,0.6721,0.0,0.0000,1.6786,0.2565,12.4,0.8218,0.4,0.5495,1.3713,0.8196,5.9367,0.3548,0.183,0.4757,0.091,0.5886,0.1680,0.3588,20.0,0.0434,1.4665,0.4965,0.0000,0.0000,0.0000,0.0000,18.9388,0.6001,0.0000,0.0000,0.0,0.0,0.0,0.0,0.6001,0.2687,93.4334,0.6296,30.4,0.2435,4.7500,0.8933,1

# I need to get rid of duplicate rows (census tracts that have multiple power plants - I need to create a new column with the number of natural gas plants per tract and then another one with all the names)

In [153]:
md.to_csv('md.csv',index=False)

In [154]:
%%R
 
# Import data with R
df_md <- read_csv('md.csv', show_col_types = FALSE)
df_md

# A tibble: 1,526 × 199
   STATEFP COUNTYFP TRACTCE AFFGEOID         GEOID GEOID_2020 COUNTY_x StateDesc
     <dbl>    <dbl>   <dbl> <chr>            <dbl>      <dbl> <chr>    <chr>    
 1      24        1     100 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 2      24        1     200 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 3      24        1     200 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 4      24        1     500 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 5      24        1     600 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 6      24        1     700 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 7      24        1     800 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 8      24        1    1000 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
 9      24        1    1100 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
10      24        1    1200 140000US24001… 2.40e10    2.40e10 Allegan… Maryland 
# ℹ 

In addition: Warning message:
One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat) 


In [155]:
%%R

df_md %>%
  group_by(Category_Number) %>%
  summarise(median_E_POV200 = median(E_POV200, na.rm = TRUE)) %>%
  arrange(desc(median_E_POV200))

# A tibble: 10 × 2
   Category_Number median_E_POV200
             <dbl>           <dbl>
 1               3            37.5
 2               5            29.4
 3               6            22.7
 4               1            20.7
 5               8            20.4
 6               0            19.5
 7               4            16.7
 8               9            13.8
 9               2            12.5
10               7            11.7


In [156]:
%%R

df_md %>%
  group_by(Category_Number) %>%
  summarise(median_ASTHMA = median(E_ASTHMA, na.rm = TRUE)) %>%
  arrange(desc(median_ASTHMA))

# A tibble: 10 × 2
   Category_Number median_ASTHMA
             <dbl>         <dbl>
 1               3          11.7
 2               5          11.3
 3               6          11.3
 4               8          11.2
 5               9          11.1
 6               1          11  
 7               2          11  
 8               0          10.9
 9               4          10.8
10               7          10.8


In [157]:
md['natural_gas'].value_counts()

natural_gas
0    1515
1      11
Name: count, dtype: int64

In [158]:
md['county'] = md['COUNTY_x']

In [160]:
%%R -i md
# Logistic regression with county fixed effects
model <- feglm(
  natgas_noplant ~ E_POV200 + E_AFAM | county, 
  data = md, 
  family = binomial("logit")
)

summary(model)

GLM estimation, family = binomial, Dep. Var.: natgas_noplant
Observations: 770
Fixed-effects: county: 7
Standard-errors: Clustered (county) 
          Estimate Std. Error   z value   Pr(>|z|)    
E_POV200 -0.096465   0.027783 -3.472072 0.00051646 ***
E_AFAM    0.011240   0.020719  0.542526 0.58745619    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Log-Likelihood: -50.5   Adj. Pseudo R2: -0.014809
           BIC: 160.8     Squared Cor.:  0.035658


NOTES: 183 observations removed because of NA values (LHS: 177, RHS: 9).
       17 fixed-effects (573 observations) removed because of only 0 (or only 1) outcomes.


In [133]:
%%R -i md
logistic <- glm(natural_gas ~ E_POV200, data=md, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ E_POV200, family = "binomial", data = md)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -3.94466    0.52192  -7.558 4.09e-14 ***
E_POV200    -0.05529    0.03043  -1.817   0.0693 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 130.31  on 1516  degrees of freedom
Residual deviance: 125.83  on 1515  degrees of freedom
  (9 observations deleted due to missingness)
AIC: 129.83

Number of Fisher Scoring iterations: 8



In [134]:
%%R

PseudoR2(logistic, which='McFadden')

  McFadden 
0.03436401 


### SPL_SVM_DOM4 - Domain consisting of mobile homes and housing with group quarters

In [135]:
%%R -i md
logistic <- glm(natural_gas ~ SPL_SVM_DOM4, data=md, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ SPL_SVM_DOM4, family = "binomial", 
    data = md)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -5.8851     0.6643  -8.860   <2e-16 ***
SPL_SVM_DOM4   1.2845     0.6709   1.914   0.0556 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 130.31  on 1516  degrees of freedom
Residual deviance: 126.62  on 1515  degrees of freedom
  (9 observations deleted due to missingness)
AIC: 130.62

Number of Fisher Scoring iterations: 8



In [136]:
%%R

PseudoR2(logistic, which='McFadden')

  McFadden 
0.02829916 


In [137]:
%%R -i md
logistic <- glm(natural_gas ~ E_POV200 + E_AFAM + E_HISP + E_LIMENG + SPL_SVM_DOM4, data=md, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ E_POV200 + E_AFAM + E_HISP + E_LIMENG + 
    SPL_SVM_DOM4, family = "binomial", data = md)

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -5.36455    0.93479  -5.739 9.54e-09 ***
E_POV200     -0.07464    0.03354  -2.225   0.0260 *  
E_AFAM        0.01683    0.01105   1.524   0.1276    
E_HISP        0.05595    0.05948   0.941   0.3469    
E_LIMENG     -0.29467    0.24539  -1.201   0.2298    
SPL_SVM_DOM4  1.72920    0.77517   2.231   0.0257 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 130.31  on 1516  degrees of freedom
Residual deviance: 116.95  on 1511  degrees of freedom
  (9 observations deleted due to missingness)
AIC: 128.95

Number of Fisher Scoring iterations: 9



In [138]:
%%R

PseudoR2(logistic, which='McFadden')

 McFadden 
0.1025147 


In [139]:
%%R -i md
logistic <- glm(natural_gas ~ E_POV200 + E_AFAM + E_HISP + E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + E_RENTER + E_DAYPOP, data=md, family="binomial")
summary(logistic)


Call:
glm(formula = natural_gas ~ E_POV200 + E_AFAM + E_HISP + E_AIAN + 
    E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + 
    E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + 
    E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + 
    E_RENTER + E_DAYPOP, family = "binomial", data = md)

Coefficients:
               Estimate Std. Error z value Pr(>|z|)  
(Intercept)   3.155e+00  7.581e+00   0.416   0.6773  
E_POV200     -9.114e-02  6.720e-02  -1.356   0.1750  
E_AFAM        1.341e-02  1.565e-02   0.857   0.3914  
E_HISP        3.587e-02  7.897e-02   0.454   0.6497  
E_AIAN       -1.722e-01  1.230e+00  -0.140   0.8886  
E_LIMENG     -2.053e-01  2.759e-01  -0.744   0.4567  
SPL_SVM_DOM4  1.377e+00  8.564e-01   1.608   0.1078  
E_UNEMP       1.372e-01  1.178e-01   1.165   0.2440  
E_NOHSDP      1.321e-01  1.090e-01   1.212   0.2254  
E_UNINSUR    -1.635e-01  1.621e-01  -1.009   0.3131  
E_TOTPOP      1.202e-04  1.764e-04   0.681   0.4957  
E_PARK       -1

In [140]:
%%R

PseudoR2(logistic, which='McFadden')

 McFadden 
0.2922839 


In [141]:
%%R -i md
model <- lm(E_CANCER ~ natural_gas + E_POV200 + E_AFAM + E_HISP + E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + E_RENTER + E_DAYPOP, data=md)
summary(model)


Call:
lm(formula = E_CANCER ~ natural_gas + E_POV200 + E_AFAM + E_HISP + 
    E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + 
    E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + 
    E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + 
    E_RENTER + E_DAYPOP, data = md)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6012 -0.4650 -0.0169  0.5185  3.0291 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   3.649e+00  4.890e-01   7.462 1.44e-13 ***
natural_gas  -2.903e-01  2.577e-01  -1.127 0.260014    
E_POV200     -4.896e-03  3.294e-03  -1.486 0.137479    
E_AFAM       -3.716e-02  1.026e-03 -36.225  < 2e-16 ***
E_HISP       -3.890e-02  3.504e-03 -11.104  < 2e-16 ***
E_AIAN       -1.588e-01  4.726e-02  -3.360 0.000800 ***
E_LIMENG     -2.408e-02  8.267e-03  -2.913 0.003629 ** 
SPL_SVM_DOM4  2.160e-01  5.576e-02   3.873 0.000112 ***
E_UNEMP      -1.829e-03  6.788e-03  -0.270 0.787575    
E_NOHSDP     -1.221e

In [142]:
%%R -i md
model <- lm(E_CANCER ~ E_AGE65 + E_AGE17, data=md)
summary(model)


Call:
lm(formula = E_CANCER ~ E_AGE65 + E_AGE17, data = md)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.7517 -1.1842  0.0443  1.3227  4.0177 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.692875   0.217336   7.789 1.24e-14 ***
E_AGE65     0.258748   0.005651  45.786  < 2e-16 ***
E_AGE17     0.066837   0.007298   9.158  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.613 on 1516 degrees of freedom
  (7 observations deleted due to missingness)
Multiple R-squared:  0.592,	Adjusted R-squared:  0.5915 
F-statistic:  1100 on 2 and 1516 DF,  p-value: < 2.2e-16



In [143]:
%%R -i md
model <- lm(E_ASTHMA ~ natural_gas, data=md)
summary(model)


Call:
lm(formula = E_ASTHMA ~ natural_gas, data = md)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.5551 -0.9551 -0.1551  0.7449  7.1449 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 11.05511    0.03642 303.567   <2e-16 ***
natural_gas -0.30056    0.42795  -0.702    0.483    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.414 on 1517 degrees of freedom
  (7 observations deleted due to missingness)
Multiple R-squared:  0.0003251,	Adjusted R-squared:  -0.0003339 
F-statistic: 0.4933 on 1 and 1517 DF,  p-value: 0.4826



In [144]:
%%R -i md
model <- lm(E_ASTHMA ~ natural_gas + E_POV200 + E_AFAM + E_HISP + E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + E_RENTER + E_DAYPOP, data=md)
summary(model)


Call:
lm(formula = E_ASTHMA ~ natural_gas + E_POV200 + E_AFAM + E_HISP + 
    E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + 
    E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + 
    E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + 
    E_RENTER + E_DAYPOP, data = md)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.75548 -0.38783  0.01452  0.39132  1.88346 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   8.793e+00  3.455e-01  25.446  < 2e-16 ***
natural_gas   6.567e-02  1.821e-01   0.361 0.718406    
E_POV200      4.524e-02  2.328e-03  19.433  < 2e-16 ***
E_AFAM        1.406e-02  7.249e-04  19.398  < 2e-16 ***
E_HISP       -2.592e-02  2.476e-03 -10.470  < 2e-16 ***
E_AIAN       -3.692e-02  3.339e-02  -1.106 0.269028    
E_LIMENG     -6.049e-02  5.842e-03 -10.355  < 2e-16 ***
SPL_SVM_DOM4  4.734e-02  3.940e-02   1.201 0.229768    
E_UNEMP       7.458e-03  4.797e-03   1.555 0.120214    
E_NOHSDP  

In [145]:
%%R -i md
model <- lm(E_CHD ~ natural_gas, data=md)
summary(model)


Call:
lm(formula = E_CHD ~ natural_gas, data = md)

Residuals:
    Min      1Q  Median      3Q     Max 
-5.2071 -1.1071 -0.2071  0.9929 11.5929 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)   6.1071     0.0445 137.246   <2e-16 ***
natural_gas  -1.1253     0.5229  -2.152   0.0316 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.728 on 1517 degrees of freedom
  (7 observations deleted due to missingness)
Multiple R-squared:  0.003044,	Adjusted R-squared:  0.002386 
F-statistic: 4.631 on 1 and 1517 DF,  p-value: 0.03155



In [146]:
%%R -i md
model <- lm(E_CHD ~ E_AGE65 + E_AGE17, data=md)
summary(model)


Call:
lm(formula = E_CHD ~ E_AGE65 + E_AGE17, data = md)

Residuals:
    Min      1Q  Median      3Q     Max 
-3.2557 -0.9156 -0.2098  0.7253  6.2453 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 2.427621   0.173008  14.032   <2e-16 ***
E_AGE65     0.156181   0.004499  34.718   <2e-16 ***
E_AGE17     0.050567   0.005810   8.704   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.284 on 1516 degrees of freedom
  (7 observations deleted due to missingness)
Multiple R-squared:  0.4497,	Adjusted R-squared:  0.449 
F-statistic: 619.4 on 2 and 1516 DF,  p-value: < 2.2e-16



In [147]:
%%R -i md
model <- lm(E_CHD ~ natural_gas + E_POV200 + E_AFAM + E_HISP + E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + E_RENTER + E_DAYPOP, data=md)
summary(model)


Call:
lm(formula = E_CHD ~ natural_gas + E_POV200 + E_AFAM + E_HISP + 
    E_AIAN + E_LIMENG + SPL_SVM_DOM4 + E_UNEMP + E_NOHSDP + E_UNINSUR + 
    E_TOTPOP + E_PARK + E_AGE17 + E_AGE65 + EPL_ROAD + E_RAIL + 
    E_AIRPRT + EPL_IMPWTR + EPL_SMOKE + E_NOINT + E_HOUBDN + 
    E_RENTER + E_DAYPOP, data = md)

Residuals:
    Min      1Q  Median      3Q     Max 
-4.7495 -0.4270  0.0152  0.4308  3.5948 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1.617e+00  4.305e-01   3.757 0.000179 ***
natural_gas  -1.250e-01  2.269e-01  -0.551 0.581769    
E_POV200      4.738e-02  2.900e-03  16.335  < 2e-16 ***
E_AFAM       -4.557e-03  9.031e-04  -5.047 5.05e-07 ***
E_HISP       -2.297e-02  3.084e-03  -7.446 1.62e-13 ***
E_AIAN       -5.811e-02  4.160e-02  -1.397 0.162693    
E_LIMENG     -3.280e-02  7.278e-03  -4.507 7.10e-06 ***
SPL_SVM_DOM4  3.154e-01  4.908e-02   6.427 1.75e-10 ***
E_UNEMP       1.863e-02  5.976e-03   3.117 0.001860 ** 
E_NOHSDP      4.781e-02